In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
        Info Given MorphoMol was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
  ? MorphoMol
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83].


In [2]:
@load "../../Data/jld2/2_experimental/2_6r7m_protor.jld2"

3-element Vector{Symbol}:
 :template_mol
 :template_radii
 :x_init

In [6]:
experimental_templates = Dict()
folder = "../../Data/jld2/2_experimental/"
for file in readdir(folder)
    mol_type = split(file, "_")[2]
    @load "$(folder)$(file)" template_mol template_radii x_init
    experimental_templates[mol_type] = Dict("template_mol" => template_mol, "template_radii" => template_radii, "state" => x_init)
end

In [12]:
open("2su_experimental_assembly.jl", "w") do file
    write(file, "2TMVSU_EXPERIMENTAL_ASSEMBLY = $(experimental_templates)")
end

217243